In [1]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece

import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm

1.0.2
2.8.0
4.8.2
0.17.0


In [2]:
!pip install transformers==4.8.2
!pip install tensorflow_addons

import pandas as pd
import numpy as np

import re
import os
import pickle 
import dill # for saving a function as a file
import logging # for changing the tf's logging level
from IPython.display import clear_output 

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, initializers, losses, optimizers, metrics 

import transformers
from transformers import TFBertModel 

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

# clear the output after the installation
clear_output() 

In [3]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

In [4]:
from tqdm import tqdm

In [5]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [6]:
from google.colab import drive
drive.mount('/gdrive')

#경로 수정 필수
data_path = '/gdrive/MyDrive/colab_data_3/temp_data/' 


# 1) Load the Model-builder (function)
with open(data_path + 'model_BERTfunction_v1.pkl', 'rb') as f:
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
with open(data_path + 'tokenizer-bert.pkl', 'rb') as f:
    tokenizer = pickle.load(f) 


# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=300) 
    
#경로 수정 필수
checkpoint_path = '/gdrive/MyDrive/colab_data_3/temp_data/saved_models/'
model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [7]:

def predict_sentiment(sentence, tokenizer, model):
    
    SEQ_LEN = 300 # 최대 token 개수 이상의 값으로 임의로 설정

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence),
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) # SEQ_LEN == 128
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) # shape == (1, 128) : like appending to a list 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    prediction = model.predict(new_inputs)
    predicted_probability = np.round(np.max(prediction) * 100, 2) # ex) [[0.0125517 0.9874483]] -> round(0.9874483 * 100, 2) -> round(98.74483, 2) -> 98.74
    predicted_class = ["분노", "놀람", "행복", "슬픔", "혐오","공포" ][np.argmax(prediction, axis=1)[0]] # ex) ['부정', '긍정'][[1][0]] -> ['부정', '긍정'][1] -> '긍정'
    
    #태깅용
    return predicted_class, predicted_probability 
    
    ##결과확인용
    #print("{}% 확률로 {} 음악입니다.".format(predicted_probability, predicted_class)) 

In [8]:
# load merged dataset
from google.colab import files
file_uploaded = files.upload()

Saving allsong_processed.csv to allsong_processed.csv


In [9]:
df = pd.read_csv("allsong_processed.csv", encoding='utf-8', lineterminator='\n', index_col=0);df

,index,Genre,Artist,Title,Lyric,Date
0,0,"발라드, 국내드라마",멜로망스,사랑인가 봐,너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보...,2022.02.18
1,1,발라드,경서,나의 X에게,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,2022.04.24
2,2,발라드,김민석 (멜로망스),취중고백,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,2021.12.19
3,3,"발라드, 국내드라마",임영웅,사랑은 늘 도망가,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,2021.10.11
4,4,발라드,MSG워너비(M.O.M),듣고 싶을까,우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌...,2022.02.26
...,...,...,...,...,...,...
3569,3995,성인가요/트로트,조항조,만약에,만약에 당신이 그 누구와 사랑에 빠지면 그 사랑을 위해서 무얼할 수 있나 텅빈세상 ...,2020.07.31
3570,3996,성인가요/트로트,송가인,용두산 엘레지,용두산아 용두산아너만은 변치 말자한 발 올려 맹세하고두 발 디뎌 언약하던한 계단 두...,2021.03.21
3571,3997,성인가요/트로트,송가인,단장의 미아리 고개,미아리 눈물고개임이 넘던 이별 고개화약 연기 앞을 가려눈 못 뜨고 헤매일 때당신은 ...,2020.12.26
3572,3998,성인가요/트로트,조항조,때,낮춰야 할 때는 낮춰야하고 물러나야 할 때는 물러나야지 오르고 오르면 끝이 있듯이 ...,2016.07.13


# 곡 감정 태깅

In [11]:
for row in tqdm(df.iterrows()):
  # #장르 
  # print(row[1][1])
  # #아티스트 
  # print(row[1][2])
  # #타이틀 
  # print(row[1][3])
  # #가사 
  print(row[1][4])
  # #발매일 
  # print(row[1][5])
  # print(row[1][-1])
  #print(row)
  break
  #predict_sentiment(row[1][-1], tokenizer, model)
  

0it [00:00, ?it/s]

너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보고 있으면자연스레 따라 웃고 있는 걸너의 행동에 설레어하고 뒤척이다가지새운 밤이 많아지는데이건 누가 봐도 사랑일 텐데종일 함께면 질릴 텐데나 돌아서도 온통 너인 건아무래도 사랑인가 봐점점 너와 하고 싶은 일들 생각하면서하룰 보낸 날이 많아지는데이건 누가 봐도 사랑일 텐데종일 함께면 질릴 텐데나 돌아서도 온통 너인 건아무래도 사랑인가 봐너의 행복해하는 모습을 보고 있으면나도 모르게 따라 웃는데이 정도면 알아줄 만하잖아너도 용기 낼만 하잖아나만 이런 게 아니라면우리 만나볼 만하잖아아무래도 사랑인가 봐


In [12]:
Emotion = []
Probability = []

for row in tqdm(df.iterrows()):
  predicted_class, predicted_probability = predict_sentiment(row[1][4], tokenizer, model)
  #break
  Emotion.append(predicted_class)
  Probability.append(predicted_probability)

0it [00:00, ?it/s]INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>]
3574it [36:28,  1.63it/s]


In [13]:
df['Emotion'] = Emotion
df['Proba'] = Probability

In [14]:
df

,index,Genre,Artist,Title,Lyric,Date,Emotion,Proba
0,0,"발라드, 국내드라마",멜로망스,사랑인가 봐,너와 함께 하고 싶은 일들을상상하는 게요즘 내 일상이 되고너의 즐거워하는 모습을 보...,2022.02.18,슬픔,64.12
1,1,발라드,경서,나의 X에게,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,2022.04.24,슬픔,55.27
2,2,발라드,김민석 (멜로망스),취중고백,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,2021.12.19,슬픔,70.52
3,3,"발라드, 국내드라마",임영웅,사랑은 늘 도망가,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,2021.10.11,슬픔,79.44
4,4,발라드,MSG워너비(M.O.M),듣고 싶을까,우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌...,2022.02.26,슬픔,68.81
...,...,...,...,...,...,...,...,...
3569,3995,성인가요/트로트,조항조,만약에,만약에 당신이 그 누구와 사랑에 빠지면 그 사랑을 위해서 무얼할 수 있나 텅빈세상 ...,2020.07.31,슬픔,62.06
3570,3996,성인가요/트로트,송가인,용두산 엘레지,용두산아 용두산아너만은 변치 말자한 발 올려 맹세하고두 발 디뎌 언약하던한 계단 두...,2021.03.21,슬픔,44.35
3571,3997,성인가요/트로트,송가인,단장의 미아리 고개,미아리 눈물고개임이 넘던 이별 고개화약 연기 앞을 가려눈 못 뜨고 헤매일 때당신은 ...,2020.12.26,슬픔,48.68
3572,3998,성인가요/트로트,조항조,때,낮춰야 할 때는 낮춰야하고 물러나야 할 때는 물러나야지 오르고 오르면 끝이 있듯이 ...,2016.07.13,슬픔,66.13


In [16]:
df.to_csv("song_tagged_2.csv", encoding='utf-8')

In [18]:
df['Emotion'].value_counts()

슬픔    2419
분노     782
행복     298
놀람      75
Name: Emotion, dtype: int64